In [8]:
import opendatasets as od
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1. Open the data in PySpark

In [2]:
# download dataset from kaggle, API key required!
od.download('https://www.kaggle.com/datasets/ankurnapa/brewery-operations-and-market-analysis-dataset/data?select=brewery_data_complete_extended.csv')

Skipping, found downloaded files in ".\brewery-operations-and-market-analysis-dataset" (use force=True to force download)


In [3]:
data_path = "brewery-operations-and-market-analysis-dataset/brewery_data_complete_extended.csv"

# create a spark session
spark = SparkSession.builder.appName("Brewery Data").getOrCreate()
# enable arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# read kaggle csv file into spark dataframe
# header=True takes the first row as our header
# inferSchema=True let's spark assign variable types to our data e.g. if 
# our data contains integers spark would normally treat it as a string, but 
# with inferSchema set to True it will treat it as an integer and so on
df = spark.read.csv(data_path, header=True, inferSchema=True)
df.show()


+--------+-------------------+----------+-------+---------------+-----------------+------------------+------------------+------------------+-----------------+----------+-----+----------------+---------------+------------------+-----------------+--------------------+-------------------+------------------------+----------------------------+
|Batch_ID|          Brew_Date|Beer_Style|    SKU|       Location|Fermentation_Time|       Temperature|          pH_Level|           Gravity|  Alcohol_Content|Bitterness|Color|Ingredient_Ratio|Volume_Produced|       Total_Sales|    Quality_Score|Brewhouse_Efficiency|Loss_During_Brewing|Loss_During_Fermentation|Loss_During_Bottling_Kegging|
+--------+-------------------+----------+-------+---------------+-----------------+------------------+------------------+------------------+-----------------+----------+-----+----------------+---------------+------------------+-----------------+--------------------+-------------------+------------------------+-------

# 2. Explore the data

In [4]:
# let's look at our column names and type
# effect of setting inferSchema=True
df.printSchema()

root
 |-- Batch_ID: integer (nullable = true)
 |-- Brew_Date: timestamp (nullable = true)
 |-- Beer_Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Fermentation_Time: integer (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- pH_Level: double (nullable = true)
 |-- Gravity: double (nullable = true)
 |-- Alcohol_Content: double (nullable = true)
 |-- Bitterness: integer (nullable = true)
 |-- Color: integer (nullable = true)
 |-- Ingredient_Ratio: string (nullable = true)
 |-- Volume_Produced: integer (nullable = true)
 |-- Total_Sales: double (nullable = true)
 |-- Quality_Score: double (nullable = true)
 |-- Brewhouse_Efficiency: double (nullable = true)
 |-- Loss_During_Brewing: double (nullable = true)
 |-- Loss_During_Fermentation: double (nullable = true)
 |-- Loss_During_Bottling_Kegging: double (nullable = true)



We need to understand some more details about our dataset, like why is color an integer?? &#128512; (Clearly I'm not a beer expert...)

- **Batch_ID (int)**: 

    Brewery-defined ID for a beer batch
- **Brew_Date (timestamp)**: 

    Date and time of brewing given as YYYY-MM-DD hh:mm:ss
- **Beer_Style (str)**: 

    The type of beer (e.g., Wheat Beer, Ale, Stout, etc.)
- **SKU (str)**: 

    Type of packaging (stock keeping unit) (e.g. Kegs, Cans, etc.)
- **Location (str)**: 

    Name of an Indian city where the beer is produced.
- **Fermentation_Time (int)**: 

    Fermentation time measured in days.
- **Temperature (double)**: 

    Average temperature (°C) maintained in the brewing process. There are a surprisingly large number of decimal units (I will cut this off to the first decimal place).
- **pH_Level (double)**: 

    Indicates the acidity or alkalinity of the beer. There are a surprisingly large number of decimal units (I will cut this off to the first decimal place).
- **Gravity (double)**: 

    Method of estimating potential alcohol content by taking the ratio of beer density over water density (unitless). There are a surprisingly large number of decimal units (I will cut this off to the third decimal place).
- **Alcohol_Content (double)**: 

    Percentage of alcogol by volume in the beer. There are a surprisingly large number of decimal units (I will cut this off to the first decimal place).
- **Bitterness (int)**: 

    Bitterness of the beer based on IBU (International Bitterness Units).
- **Color (int)**: 

    Color of the beer based on SRM (Standard Reference Method)
- **Ingredient_Ratio (str)**: 

    Ratio of ingredients used in the beer (no additional information provided).
- **Volume_Produced (integer)**: 

    The liters of beer produced in the batch.
- **Total_Sales (double)**: 

    The total sales generated from the batch (I suppose in Rupees, but will have look into that). There are a surprisingly large number of decimal units (I will cut this off to the second decimal place).
- **Quality_Score (double)**: 

    Overall quality score assigned to the beer batch from a scale of 0-10. There are a surprisingly large number of decimal units (I will cut this off to the first decimal place).
- **Brewhouse_Efficiency (double)**: 

    Efficiency of the brewing process (%). (Measure of how much sugar was extracted over the potential sugar extraction amount.) There are a surprisingly large number of decimal units (I will cut this off to the second decimal place).
- **Loss_During_Brewing (double)**: 

    Percentage of volume loss during the brewing process. There are a surprisingly large number of decimal units (I will cut this off to the second decimal place).
- **Loss_During_Fermentation (double)**: 

    Percentage of volume loss during the fermentation process. There are a surprisingly large number of decimal units (I will cut this off to the second decimal place).
- **Loss_During_Bottling_Kegging (double)**: 

    Percentage of volume loss during the bottling or kegging process. There are a surprisingly large number of decimal units (I will cut this off to the second decimal place).

In [5]:
# number of columns and data points (rows)
num_columns = len(df.columns)
num_rows = df.count()

print(f"The dataset consists of {num_columns} columns and {num_rows:,} rows (batches).")


The dataset consists of 20 columns and 10,000,000 rows (batches).


In [6]:
# let's see if we have missing data (e.g. NaN)
# .describe() will go through the data, and if the count is not matching to the 
# number of rows, we know that there is data missing.
df.describe().show()

+-------+------------------+----------+--------+---------------+-----------------+------------------+-------------------+--------------------+-------------------+------------------+-----------------+----------------+-----------------+------------------+------------------+--------------------+-------------------+------------------------+----------------------------+
|summary|          Batch_ID|Beer_Style|     SKU|       Location|Fermentation_Time|       Temperature|           pH_Level|             Gravity|    Alcohol_Content|        Bitterness|            Color|Ingredient_Ratio|  Volume_Produced|       Total_Sales|     Quality_Score|Brewhouse_Efficiency|Loss_During_Brewing|Loss_During_Fermentation|Loss_During_Bottling_Kegging|
+-------+------------------+----------+--------+---------------+-----------------+------------------+-------------------+--------------------+-------------------+------------------+-----------------+----------------+-----------------+------------------+-----------

Interestingly, .describe() attempts at giving us simple statistics on string columns, but manages to ignore the timestamp column (Brew_Date). Let's have a look at the Brew_Date to make sure that we don't overlook missing data there.

In [11]:
brew_date = df.columns[1]
null_count = df.filter(F.col(brew_date).isNull()).count()
print(f"There are {null_count} missing data points in the Brew Date column")

There are 0 missing data points in the Brew Date column
